In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import ResNet50, EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB7, VGG16, ResNet50V2, InceptionV3
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pandas as pd
import imgaug as ia
import imgaug.augmenters as iaa
import imageio
import glob, os
from pathlib import Path
import matplotlib.pyplot as plt
import logging
import tensorflow_addons as tfa
from tensorflow.keras import mixed_precision

TPU=False

In [ ]:
if TPU:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU')) 
else:
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
    tf.config.list_physical_devices('GPU')

In [ ]:
# enable XLA 
tf.config.optimizer.set_jit(True)

In [ ]:
config = {
    'model': 'EfficientNetB5',
    'img_shape': (456, 456),
    'max_epochs': 50,
    'max_epochs_per_fit': 10,
    'num_classes': 5,
    'batch_size': 32,
    'transfer_learning': True,
    'pre_training_epochs': 10,
    'pre_training_only': False,
    'pre_training_learning_rate': 5e-2,
    'pre_training_min_delta': 0.01,
    'pre_training_patience': 2,
    'fine_tuning_learning_rate': 1e-4,
    'fine_tuning_min_delta': 0.01,
    'fine_tuning_patience': 2,
    'fine_tuning_unfreeze_interval': 15,
    'f1_average': 'macro',
    'f1_threshold': 0.7,
    'validation_split': 0.15,
    'csv_location': '../input/plant-pathology-fgvc78-640px/train.csv',
    'data_location': '../input/plant-pathology-fgvc78-640px/train_images',
    'gamma': 0.3 # for label smoothing
}
model_map = {
    'EfficientNetB0': EfficientNetB0,
    'EfficientNetB1': EfficientNetB1,
    'EfficientNetB2': EfficientNetB2,
    'EfficientNetB3': EfficientNetB3,
    'EfficientNetB4': EfficientNetB4,
    'EfficientNetB5': EfficientNetB5,
    'ResNet50': ResNet50,
    'ResNet50V2': ResNet50V2,
    'InceptionV3': InceptionV3
}

In [ ]:
# labels need to be converted to a list of labels (space separator)
df = pd.read_csv(config['csv_location'], delimiter=',')
make_multi_label = lambda x: x.split(' ')
df.labels = df.labels.apply(make_multi_label)

In [ ]:
# EXPERIMENTAL: remove healthy label
df.labels = df.labels.apply(lambda x: [item for item in x if item != 'healthy'])
df

In [ ]:
# check if dataframe contains multiple labels
df

In [ ]:
##augmentation test 
#class AugmenterConfig:
#    name: str = 'basic'
#    extension: str = 'default'
#    width: int = 224
#    height: int = 224
#    pad_mode: str = 'edge'
#    position: str = 'center'
#        
#def basic_augmenter(cfg: AugmenterConfig):
#    return iaa.Sequential([
#        iaa.Resize({'shorter-side': 'keep-aspect-ratio', 'longer-side': max(cfg.width, cfg.height)}),
#       # iaa.CropToSquare(position='center')
#        iaa.PadToFixedSize(width=cfg.width, height=cfg.height, position=cfg.position, pad_mode=cfg.pad_mode, pad_cval=(0, 0))
#    ])
#
#cfg = AugmenterConfig()
#%matplotlib inline 
#aug = basic_augmenter(cfg)
#path='../input/plant-pathology-2021-fgvc8/train_images/803553d5df59cac2.jpg'
#img = imageio.imread(path)
#new_img = aug(images=[img])[0]
#plt.imshow(new_img)

In [ ]:
## images need to be cropped offline to speed up training by factor of ~50
#cfg = AugmenterConfig()
#aug = basic_augmenter(cfg)
#src_path = Path('../input/plant-pathology-2021-fgvc8/train_images')
#dst_path = Path('./train_images')
#dst_path.mkdir(exist_ok=True)
#for file in glob.glob(f'{src_path}/*.jpg'):
#    new_img = aug(images=[img])[0]
#    pf = Path(file)
#    imageio.imwrite(dst_path / f'{pf.stem}{pf.suffix}', new_img)

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, test_size=config['validation_split'])

In [ ]:
data_dir = config['data_location']

train_datagen = ImageDataGenerator()  # rescale=1. / 255)
train_generator = train_datagen.flow_from_dataframe(df_train, directory=data_dir, x_col='image', y_col='labels', weight_col=None, target_size=config['img_shape'], color_mode='rgb', classes=None, class_mode='categorical', batch_size=config['batch_size'], shuffle=True, seed=None, save_to_dir=None, save_prefix='', save_format='jpg', subset=None, interpolation='nearest', validate_filenames=True)
val_datagen = ImageDataGenerator()  # rescale=1. / 255)
val_generator = train_datagen.flow_from_dataframe(df_val, directory=data_dir, x_col='image', y_col='labels', weight_col=None, target_size=config['img_shape'], color_mode='rgb', classes=None, class_mode='categorical', batch_size=config['batch_size'], shuffle=True, seed=None, save_to_dir=None, save_prefix='', save_format='jpg', subset=None, interpolation='nearest', validate_filenames=True)

In [ ]:
train_generator.class_indices

In [ ]:
img_augmentation = Sequential(
    [
        preprocessing.RandomRotation(factor=0.15),
        #preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        # preprocessing.RandomFlip(),
        preprocessing.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

def unfreeze_model(model, to_unfreeze=20, lr=1e-4):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-to_unfreeze:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    f1 = tfa.metrics.F1Score(num_classes=config['num_classes'], threshold=config['f1_threshold'], average=config['f1_average'])
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=config['gamma'], reduction="auto", name="binary_crossentropy")
    model.compile(
        optimizer=optimizer, loss=loss, metrics=["accuracy", f1]
    )
    return model

def generic_builder(name, net, lr=1e-2, dropout_rate=0.2, num_classes=6, img_shape=(380,380,3)):
    inputs = layers.Input(img_shape)
    x = img_augmentation(inputs)
    model = net(include_top=False, input_tensor=x, weights='imagenet')
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = dropout_rate
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    x = layers.Dense(
        num_classes, activation="sigmoid", name="pred")(x)
    outputs = tf.cast(x, tf.float32) # workaround for fp16 crossentropy bug
    
    # Compile
    model = tf.keras.Model(inputs, outputs, name=name)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    f1 = tfa.metrics.F1Score(num_classes=config['num_classes'], threshold=config['f1_threshold'], average=config['f1_average'])
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=config['gamma'], reduction="auto", name="binary_crossentropy")
    model.compile(
        optimizer=optimizer, loss=loss, metrics=["accuracy", f1]
    )
    return model

In [ ]:
if TPU:
    with strategy.scope():
        model = generic_builder(config['model'], model_map[config['model']], num_classes=config['num_classes'], img_shape=(config['img_shape'][0], config['img_shape'][1], 3))
else:
    model = generic_builder(config['model'], model_map[config['model']], num_classes=config['num_classes'], img_shape=(config['img_shape'][0], config['img_shape'][1], 3))    

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping
pre_training_early_stopping_callback = EarlyStopping(
    monitor='val_f1_score', mode='max', min_delta=config['pre_training_min_delta'], patience=config['pre_training_patience'], restore_best_weights=True)
pre_training_callbacks = [pre_training_early_stopping_callback]

fine_tuning_early_stopping_callback = EarlyStopping(
    monitor='val_f1_score', mode='max', min_delta=config['fine_tuning_min_delta'], patience=config['fine_tuning_patience'], restore_best_weights=True)
fine_tuning_callbacks = [fine_tuning_early_stopping_callback]

In [ ]:
epochs_to_date = 0
history = model.fit(train_generator, validation_data=val_generator, epochs=config['pre_training_epochs'], callbacks=pre_training_callbacks)

In [ ]:
if config['transfer_learning'] and not config['pre_training_only']:
    for to_unfreeze in range(config['fine_tuning_unfreeze_interval'], len(model.layers), config['fine_tuning_unfreeze_interval']):
        if TPU:
            with strategy.scope():
                model = unfreeze_model(model, to_unfreeze, lr=config['fine_tuning_learning_rate'])
        else:
            model = unfreeze_model(model, to_unfreeze, lr=config['fine_tuning_learning_rate'])
        trainable_layers = len(
            [1 for layer in model.layers if layer.trainable is True])
        logging.info(f'Fine-tuning on {trainable_layers}')
        epochs_to_date += len(history.history['loss'])
        if epochs_to_date >= config['max_epochs']:
            break
        epochs_to_do = epochs_to_date+config['max_epochs_per_fit'] if epochs_to_date + \
            config['max_epochs_per_fit'] < config['max_epochs'] else config['max_epochs']
        history = model.fit(train_generator, validation_data=val_generator, initial_epoch=epochs_to_date,
                            epochs=epochs_to_do, callbacks=fine_tuning_callbacks)

In [ ]:
model.save('model-best.h5')